In [1]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import spacy
import re

DIR = 'Data'
path = DIR + '/Got3.txt'
got = open(path).read()

# Author typo notice & fix:
print("Dialogs open/close marks: {}/{}".format(got.count('“'), got.count('”')))
where = 'And your hair, your golden hair...'
where_idx = got.index(where) + len(where)
got = got[:where_idx] +'”'+ got[where_idx:]
print("Dialogs open/close marks: {}/{}\n".format(got.count('“'), got.count('”')))

print(got[:300])

Dialogs open/close marks: 182/181
Dialogs open/close marks: 182/182

The king is dead, they told him, never knowing that Joffrey was his son as well as his
sovereign.
 “The Imp opened his throat with a dagger,” a costermonger declared at the roadside inn where
they spent the night. “He drank his blood from a big gold chalice.” The man did not recognize
the bearded on


In [2]:
# Initial cleaning
got = " ".join(got.split('\n'))
got = got.replace('  ', ' ')
got = got.replace('   ', ' ')
got = got.replace('...', '369')


def replace_text_with_mapped_tag(i, tag, dialogs, dialog, got):
    key = "D_"+str(i)+tag+"_D"
    dialogs[key] = dialog
    got = got.replace(dialog, key)
    return dialogs, got

punct_map = {"69":"_369", # <- 369, and last 2 digits of it stands for '...'
              ".":"_dot",   ",":"_coma",   "?":"_question",    "!":"_exc",    "-":"_dash",}

dialogs = OrderedDict(dict())
for i in range(1, got.count('“')):
    # Temporary replace dialogs with mapped tags
    open_quote, end_quote = got.index('“'), got.index('”')
    assert open_quote < end_quote
    
    used = False
    dialog = got[open_quote:end_quote + 1]
    last2_letters = dialog[-3:-1]
    for punct, tag in punct_map.items():
        if last2_letters.endswith(punct) or last2_letters[-1].isalpha():
            dialogs, got = replace_text_with_mapped_tag(i, tag, dialogs, dialog, got)
            used = True
    if not used:
        print("Other punct: _{}_ \nin: {})".format(last2_letters, dialog))
got_s = [s.strip() for s in got.split(" ") if s]


def new_sent(sent, w, sents):
    sent += " " + w
    sents.append(sent)
    return "", sents

def checkEnds(line, ends):
    return any(line.endswith(end) for end in ends)

sents = []
sent = ""
for i, w in enumerate(got_s):
    # Glue lines back, word by word, according to new rules
    if i+1 < len(got_s):
        if w.endswith('_D'):
            if 'coma' in w or got_s[i+1].islower():
                sent += " " + w
            else:
                sent, sents = new_sent(sent, w, sents)
        elif checkEnds(w, ['!', '.', '?', "369"]) and not got_s[i+1].islower():
            sent, sents = new_sent(sent, w, sents)
        else:
            sent += " " + w
    else:
        print(i, w)
        sent, sents = new_sent(sent, w, sents)
        
remapped_got_lines = []
for sent in sents:
    # Revive dialogs back as separate or connected lines accodrding to rules planted in sents
    if 'D_' in sent:
        for w in sent.split(" "):
            if w.startswith("D_"):
                sent = sent.replace(w, dialogs[w])
    remapped_got_lines.append(sent)
        
remapped_got_lines = [line.replace('369', '...') for line in remapped_got_lines]

for i, sent in enumerate(remapped_got_lines[:]):
    print(i, ">", sent)

2840 D_181_dot_D
0 >  The king is dead, they told him, never knowing that Joffrey was his son as well as his sovereign.
1 >  “The Imp opened his throat with a dagger,” a costermonger declared at the roadside inn where they spent the night.
2 >  “He drank his blood from a big gold chalice.”
3 >  The man did not recognize the bearded one-handed knight with the big bat on his shield, no more than any of them, so he said things he might otherwise have swallowed, had he known who was listening.
4 >  “It was poison did the deed,” the innkeep insisted.
5 >  “The boy’s face turned black as a plum.”
6 >  “May the Father judge him justly,” murmured a septon.
7 >  “The dwarf’s wife did the murder with him,” swore an archer in Lord Rowan’s livery.
8 >  “Afterward, she vanished from the hall in a puff of brimstone, and a ghostly direwolf was seen prowling the Red Keep, blood dripping from his jaws.”
9 >  Jaime sat silent through it all, letting the words wash over him, a horn of ale forgotten in hi

In [9]:
dialogs

OrderedDict([('D_1_coma_D', '“The Imp opened his throat with a dagger,”'),
             ('D_2_dot_D', '“He drank his blood from a big gold chalice.”'),
             ('D_3_coma_D', '“It was poison did the deed,”'),
             ('D_4_dot_D', '“The boy’s face turned black as a plum.”'),
             ('D_5_coma_D', '“May the Father judge him justly,”'),
             ('D_6_coma_D', '“The dwarf’s wife did the murder with him,”'),
             ('D_7_dot_D',
              '“Afterward, she vanished from the hall in a puff of brimstone, and a ghostly direwolf was seen prowling the Red Keep, blood dripping from his jaws.”'),
             ('D_8_question_D', '“What’s that awful stink?”'),
             ('D_9_question_D',
              '“Smoke, sweat, and shit. King’s Landing, in short. If you have a good nose you can smell the treachery too. You’ve never smelled a city before?”'),
             ('D_10_dot_D',
              '“I smelled White Harbor. It never stank like this.”'),
             ('D_11_d

In [5]:
got

'The king is dead, they told him, never knowing that Joffrey was his son as well as his sovereign. D_1_coma_D a costermonger declared at the roadside inn where they spent the night. D_2_dot_D The man did not recognize the bearded one-handed knight with the big bat on his shield, no more than any of them, so he said things he might otherwise have swallowed, had he known who was listening. D_3_coma_D the innkeep insisted. D_4_dot_D D_5_coma_D murmured a septon. D_6_coma_D swore an archer in Lord Rowan’s livery. D_7_dot_D Jaime sat silent through it all, letting the words wash over him, a horn of ale forgotten in his one good hand. Joffrey. My blood. My firstborn. My son. He tried to bring the boy’s face to mind, but his features kept turning into Cersei’s. She will be in mourning, her hair in disarray and her eyes red from crying, her mouth trembling as she tries to speak. She will cry again when she sees me, though she’ll fight the tears. His sister seldom wept but when she was with him